In [17]:
import torch
from torchvision import models
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset
import json
import os
import time


# Load Pre-trained Model

In [18]:
resnet50 = models.resnet50(pretrained=True)


# Image Preprocessing

In [ ]:
preprocess = transforms.Compose([
  transforms.Resize(256), 
  transforms.CenterCrop(224), 
  transforms.ToTensor(), 
  transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
  )])


# Load ImageNet label

In [ ]:
# with open('./data/imagenet_classes.txt') as f:
#     labels = [ line.strip() for line in f.readlines() ]

# len(labels)


In [19]:
with open('./in_to_in9.json') as f:
    data = f.read()

in2in9_label = json.loads(data)


In [20]:
def getIn9Label(indexes):        
    return torch.tensor([in2in9_label[str(index.item())] for index in indexes])


# Load Testing Data

In [22]:
!wget -qO mixed_same.tar.gz https://www.dropbox.com/s/f2525w5aqq67kk0/mixed_same.tar.gz?dl=0
!wget -qO mixed_rand.tar.gz https://www.dropbox.com/s/cto15ceadgraur2/mixed_rand.tar.gz?dl=0
!wget -qO only_bg_t.tar.gz https://www.dropbox.com/s/03lk878q73hyjpi/only_bg_t.tar.gz?dl=0
!wget -qO original.tar.gz https://www.dropbox.com/s/0vv2qsc4ywb4z5v/original.tar.gz?dl=0


In [24]:
!tar -xf 'mixed_same.tar.gz'
!tar -xf 'mixed_rand.tar.gz'
!tar -xf 'only_bg_t.tar.gz'
!tar -xf 'original.tar.gz'

# Evaluation

In [ ]:
class TestImage(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.images = []
        self.labels = []

        image_ext = ['.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif']
        for subdir in os.listdir(img_dir):
            subpath = os.path.join(img_dir, subdir)
            if os.path.isdir(subpath):
                label, _ = subdir.split('_')
                label = int(label)
                for f in os.listdir(subpath):
                    if any(f.lower().endswith(ext) for ext in image_ext):
                        self.images.append(f)
                        self.labels.append(label)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]

        img = imread(img_path)
        if self.transform:
            img_t = self.transform(img)
            batch_t = torch.unsqueeze(img_t, 0)

        return {
            "image": batch_t,
            'label': label
        }
    
    def __len__(self):
        return len(self.label)


In [ ]:
def evaluation(test_dir):
    testset = ImageFolder(test_dir, transform=preprocess)
    testloader = DataLoader(testset, batch_size=64, shuffle=False)

    # Compute the model accuracy on the test set
    correct = 0
    total = 0

    for i, (inputs, target) in enumerate(testloader):
        inputs = inputs.cuda()
        out = resnet50(inputs)

        _, index = torch.max(out, 1)
        # percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
        # labels[index], percentage[index].item()
        total += len(target)
        correct += (getIn9Label(index) == target).sum()

    print('Testing on', test_dir, 'Accuracy: %2.2f %%' % ((100.0 * correct) / total))

In [ ]:
resnet50.eval()
resnet50 = resnet50.cuda()

eval_tests = ['./mixed_same/val', './mixed_rand/val', './only_bg_t/val', './original/val']

start = time.time()

for test_data in eval_tests:
    evaluation(test_data)

print(time.time() - start)

Testing on ./mixed_same/val Accuracy: 80.48 %
Testing on ./mixed_rand/val Accuracy: 69.77 %
Testing on ./only_bg_t/val Accuracy: 16.82 %
Testing on ./original/val Accuracy: 95.24 %
129.74640107154846
